In [55]:
import json
import ollama
import asyncio


In [6]:
ollama.pull(model="llama3")

{'status': 'success'}

In [82]:
def get_current_conditions(params):
    print(f"執行 get_current_conditions，套入參數: {params}")

def get_location(params):
    print(f"執行 get_location，，套入參數: {params}")

def get_current_stock_price(params):
    print(f"執行 get_current_stock_price，，套入參數{params}")

In [87]:
# provide a weather checking tool to the model
tools= {
    "type": "function",
    "tools": [
        {
            "name": "get_current_conditions",
            "description": "Get the current weather conditions for a specific location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g., San Francisco, CA"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["Celsius", "Fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the user's location."
                    }
                },
                "required": ["location", "unit"]
            }
        },
        {
            "name": "get_location",
            "description": "Get the list of  current supported location",
        },
        {
            "name": "get_current_stock_price",
            "description": "Get the current stock price a country",
            "parameters": {
                "type": "object",
                "properties": {
                    "country": {
                        "type": "string",
                        "description": "The country, e.g., Taiwan, USA"
                    },
                    "unit": {
                        "type": "string",
                        "description": "The currency unit, e.g., NTD for Taiwan, USD for USA, JPY for Japan"
                    }
                },
                "required": ["location", "unit"]
            }
        },
    ]
}

function_map = {
    "get_current_conditions": get_current_conditions,
    "get_location": get_location,
    "get_current_stock_price": get_current_stock_price
}

formate = '{"name": function name, "parameters": dictionary of argument name and its value}'

def tools_switch(Query):
    response = ollama.chat(
        model='llama3',
        messages=[
            {
                'role': 'system',
                'content': f"You are a helpful assistant with tool calling capabilities. When you receive a tool call response, use the output to format an answer to the orginal user question.Given the following functions, please respond with a JSON for a function call with its proper arguments that best answers the given prompt.Respond in the format {formate}. Do not use variables.{tools}"
            },
            {
                'role': 'user',
                'content': 'What is the weather in Toronto?'
            },
            {
                'role': 'assistant',
                'content': '{ "name": "get_current_conditions", "parameters": {"location": "Toronto","unit":"Celsius"}},"required": ["location", "unit"]}'
            },
            {
                'role': 'user',
                'content': Query
            },
        ],
    )
    return response

def tool_executed(content):
    print(f"LLM 選擇執行 function: {content['name']}()")
    print("-"*20)
    
    function_name = content['name']
    parameters = content.get('parameters', None)
    if function_name in function_map:
        function_map[function_name](parameters)
    else:
        print(f"No function found for name: {function_name}")

In [88]:
# Query = "What is the weather in NY?"
# Query = "Stock price in Germany?"
Query = "How many location could be selected?"

response = tools_switch(Query)
content = json.loads(response['message']['content'])

tool_executed(content)

LLM 選擇執行 function: get_location()
--------------------
執行 get_location，，套入參數: None
